In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import copy
import time
import random
import MetaTrader5 as mt5
import pandas_ta as ta
from tapy import Indicators

In [3]:
mt5.initialize()

True

In [4]:
symbol='GOLD'

maxposition=0.05
pointer=str(random.randint(1000, 9999))

In [5]:
# Get last 500 kandels 5 minutes for Symbol

def get_futures_klines(symbol, limit=1000):
    rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M5, 0, limit)
    df = pd.DataFrame(rates)
    df['Time'] = pd.to_datetime(df['time'], unit='s')
    df['Open']=df['open'].astype(float)
    df['High']=df['high'].astype(float)
    df['Low']=df['low'].astype(float)
    df['Close']=df['close'].astype(float)
    df['Volume']=df['tick_volume'].astype(float)
    df.drop(['open', 'high', 'low', 'close', 'tick_volume','spread','real_volume','time'], axis=1, inplace=True)
    return(df)

In [6]:
# Open position for Sybol with 

def open_position(symbol, s_l, lot, comment="No specific comment", magic=0):
    
    mt5.initialize()
    prt('open: '+symbol+' quantity: '+str(lot))
    # Open a buy trade
    if (s_l=='long'):
        
        point = mt5.symbol_info(symbol).point 
        price = mt5.symbol_info_tick(symbol).ask 
        request = {
            "action": mt5.TRADE_ACTION_DEAL, 
            "symbol": symbol, 
            "volume": lot, 
            "type": mt5.ORDER_TYPE_BUY, 
            "price": price, 
            "sl": price - 500 * point,
            "tp": price + 100 * point, 
            "deviation": 10, 
            "magic": magic, 
            "comment": comment, 
            "type_time": mt5.ORDER_TIME_GTC, 
            "type_filling": mt5.ORDER_FILLING_IOC, # IOC (ETH) RETURN (XAU)
        }
        result = mt5.order_send(request)
        
        request1 = {
            "action": mt5.TRADE_ACTION_DEAL, 
            "symbol": symbol, 
            "volume": lot, 
            "type": mt5.ORDER_TYPE_BUY, 
            "price": price, 
            "sl": price - 500 * point,
            "tp": price + 200 * point, 
            "deviation": 10, 
            "magic": magic, 
            "comment": comment, 
            "type_time": mt5.ORDER_TIME_GTC, 
            "type_filling": mt5.ORDER_FILLING_IOC, # IOC (ETH) RETURN (XAU)
        }
        result1 = mt5.order_send(request1)
        
        request2 = {
            "action": mt5.TRADE_ACTION_DEAL, 
            "symbol": symbol, 
            "volume": lot, 
            "type": mt5.ORDER_TYPE_BUY, 
            "price": price, 
            "sl": price - 500 * point,
            "tp": price + 300 * point, 
            "deviation": 10, 
            "magic": magic, 
            "comment": comment, 
            "type_time": mt5.ORDER_TIME_GTC, 
            "type_filling": mt5.ORDER_FILLING_IOC, # IOC (ETH) RETURN (XAU)
        }
        result2 = mt5.order_send(request2)
    
    # Open a sell trade
    if (s_l=='short'):

        point = mt5.symbol_info(symbol).point 
        price = mt5.symbol_info_tick(symbol).bid 
        request = {
            "action": mt5.TRADE_ACTION_DEAL, 
            "symbol": symbol, 
            "volume": lot, 
            "type": mt5.ORDER_TYPE_SELL, 
            "price": price, 
            "sl": price + 500 * point,
            "tp": price - 100 * point, 
            "deviation": 10, 
            "magic": magic, 
            "comment": comment, 
            "type_time": mt5.ORDER_TIME_GTC, 
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        result = mt5.order_send(request)
        
        request1 = {
            "action": mt5.TRADE_ACTION_DEAL, 
            "symbol": symbol, 
            "volume": lot, 
            "type": mt5.ORDER_TYPE_SELL, 
            "price": price, 
            "sl": price + 500 * point,
            "tp": price - 200 * point, 
            "deviation": 10, 
            "magic": magic, 
            "comment": comment, 
            "type_time": mt5.ORDER_TIME_GTC, 
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        result1 = mt5.order_send(request1)
        
        request2 = {
            "action": mt5.TRADE_ACTION_DEAL, 
            "symbol": symbol, 
            "volume": lot, 
            "type": mt5.ORDER_TYPE_SELL, 
            "price": price, 
            "sl": price + 500 * point,
            "tp": price - 300 * point, 
            "deviation": 10, 
            "magic": magic, 
            "comment": comment, 
            "type_time": mt5.ORDER_TIME_GTC, 
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        result2 = mt5.order_send(request2)
#         return result

In [7]:
# Find all opened positions
def get_opened_positions():
    information = mt5.account_info()
    positions = mt5.positions_get()
    if len(positions)!=0:
        for position in positions:
            a = position.type
            entryprice = position.price_open
            profit = position.profit
            balance = information.balance
            id_position = position.ticket
            current_price = position.price_current
            sl = position.sl
            if a==0:
                pos = 'long'
            elif a==1:
                pos = 'short'
            yield ([pos,profit,balance,entryprice,id_position,current_price,sl])
    else:
        pos = ''
        yield ([pos])

In [8]:
def check_and_close_orders(symbol):
    positions = mt5.positions_get(symbol=symbol)
    for position in positions:
        close_position(position)

In [9]:
def get_symbol_price(symbol):
    price_1 = mt5.symbol_info_tick(symbol).bid
    price_2 = mt5.symbol_info_tick(symbol).ask
    return (price_1+price_2)/2

In [10]:
# Changing stop loss
def change_stop_loss(symbol):
    
    for position in get_opened_positions():
        
        
        if position[0]=='long':
            # Difference between current price and sl
            dif = position[5] - position[6]
            if position[6]<position[3]:
                if (position[3]+1.5)<position[5]: # position[5] - curret price
                    request = {
                        "action": mt5.TRADE_ACTION_SLTP,
                        "position": position[4],
                        "symbol" : symbol,
                        "sl": position[3]+0.5,
                    }
                    result_SL = mt5.order_send(request)
            else:    
                if dif>2:
                    request = {
                        "action": mt5.TRADE_ACTION_SLTP,
                        "position": position[4],
                        "symbol" : symbol,
                        "sl": position[6]+1,
                    }
                    result_SL = mt5.order_send(request)
                
        if position[0]=='short':
            # Difference between current price and sl
            dif = position[5] - position[6]
            if position[6]>position[3]:
                if (position[3]-1.5)>position[5]:
                    request = {
                        "action": mt5.TRADE_ACTION_SLTP,
                        "position": position[4],
                        "symbol" : symbol,
                        "sl": position[3]-0.5,
                    }
                    result_SL = mt5.order_send(request)
                
            if dif<-2:
                request = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "position": position[4],
                    "symbol" : symbol,
                    "sl": position[6]-1,
                }
                result_SL = mt5.order_send(request)

In [11]:
# generate data frame with all needed data

def PrepareDF(df):
    i = Indicators(df)
    i.alligator(period_jaws=31, period_teeth=15, period_lips=3, shift_jaws=15, shift_teeth=3, shift_lips=1, column_name_jaws='alligator_jaw', column_name_teeth='alligator_teeth', column_name_lips='alligator_lips')
    i.sma(period=9,apply_to='Close')
    i.ema(period=50,apply_to='Close')
    df = i.df
    df['rsi'] = round(ta.rsi(df['Close'], length = 14),3)
    df['alligator_jaw'] = round(df['alligator_jaw'],3)
    df['alligator_teeth'] = round(df['alligator_teeth'],3)
    df['alligator_lips'] = round(df['alligator_lips'],3)
    df['sma'] = round(df['sma'],3)
    df['ema'] = round(df['ema'],3)
    return(df)

In [12]:
def check_if_signal(symbol):
    ohlc = get_futures_klines(symbol)
    prepared_df = PrepareDF(ohlc)
    signal="" # return value
    
    i=998 
    
    if prepared_df['alligator_teeth'][i]>prepared_df['alligator_teeth'][i-1]:
        if prepared_df['alligator_teeth'][i-2]>prepared_df['alligator_teeth'][i-1]:
            if prepared_df['sma'][i]>prepared_df['ema'][i]:
                if prepared_df['rsi'][i]>50: #and prepared_df['rsi'][i]>prepared_df['rsi'][i-1]:
                    signal='long'

    if prepared_df['alligator_teeth'][i]<prepared_df['alligator_teeth'][i-1]:
        if prepared_df['alligator_teeth'][i-2]<prepared_df['alligator_teeth'][i-1]:
            if prepared_df['sma'][i]<prepared_df['ema'][i]:
                if prepared_df['rsi'][i]<50: #and prepared_df['rsi'][i]<prepared_df['rsi'][i-1]:
                    signal='short'

    return signal

In [13]:
def prt(message):
    # telegram message
    print(pointer+': '+message)

In [14]:
def main(step):

    try:
        positions=get_opened_positions()
        for position in positions:
            open_sl=position[0]
            if open_sl=="": # no position
                prt('Нет открытых позиций')
                # close all stop loss orders
                check_and_close_orders(symbol)
                signal=check_if_signal(symbol)

                if signal=='long':
                    open_position(symbol,'long',maxposition)

                elif signal=='short':
                    open_position(symbol,'short',maxposition)
            else:

                quantity=position[1]
                prt('Найдена открытая позиция '+open_sl)
                prt('Кол-во: '+str(quantity))
      
    except :
        prt('\n\nSomething went wrong. Continuing...')

In [ ]:
starttime=time.time()
timeout = time.time() + 60*60*12  # 60 seconds times 60 meaning the script will run for 12 hr
counterr=1

while time.time() <= timeout:
    try:
        prt("script continue running at "+time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
        main(counterr)
        change_stop_loss(symbol)
        counterr=counterr+1
        if counterr>5:
            counterr=1
        time.sleep(20 - ((time.time() - starttime) % 20.0)) # 1 minute interval between each new execution
    except KeyboardInterrupt:
        print('\n\KeyboardInterrupt. Stopping.')
        exit()

3824: script continue running at 2023-01-10 23:25:11
3824: Нет открытых позиций
3824: script continue running at 2023-01-10 23:25:31
3824: Нет открытых позиций
3824: script continue running at 2023-01-10 23:25:51
3824: Нет открытых позиций
3824: script continue running at 2023-01-10 23:26:11
3824: Нет открытых позиций
3824: script continue running at 2023-01-10 23:26:31
3824: Нет открытых позиций
3824: script continue running at 2023-01-10 23:26:51
3824: Нет открытых позиций
3824: script continue running at 2023-01-10 23:27:11
3824: Нет открытых позиций
3824: script continue running at 2023-01-10 23:27:31
3824: Нет открытых позиций
